<a href="https://colab.research.google.com/github/dbpedia/RDF2text-GAN/blob/master/discriminator_pretraining_(multi_input_model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pre-train Discriminator before adverserial training

### Update :
The discriminator is trained using two input layers, one for the rdf sequence, and one for the corresponding text sequence, per training instance. The idea here is that the discriminator sees two sequences and learns to predict whether or not they are related to each other, in terms of content conveyed. 


- Adapted from : Original Author:** [Apoorv Nandan](https://twitter.com/NandanApoorv)

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import time
import numpy as np




In [2]:

from google.colab import drive
drive.mount('/content/gdrive')

file_path = "/content/gdrive/My Drive/f_data.txt"
test_path = "/content/gdrive/My Drive/data/processed_graphs/eng/gat/test_data.txt"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import pretraining
from pretraining import *

In [36]:
train_dataset, tokenizer_txt = create_generator_dataset(file_path)
Xr_train, Xr_test, Xt_train, Xt_test, y_train, y_test = create_discriminator_dataset_2(train_dataset)

Max source length :  133 
Max target length :  87
Max RDF sequence size:  135
Max text sequence size:  135
dataset shape:  (68704, 135) (68704, 135) (68704, 1)


In [50]:
def probe(n, split='train'):
    if split=='test':
        print(decode_text(Xr_train[n], tokenizer_txt))
        print(decode_text(Xt_train[n], tokenizer_txt))
        print(y_train[n])
    else:
        print(decode_text(Xr_test[n], tokenizer_txt))
        print(decode_text(Xt_test[n], tokenizer_txt))
        print(y_test[n])

probe(251, 'test')

<start> <triple> acharya institute of technology affiliation visvesvaraya technological university </triple> <triple> acharya institute of technology was given the 'technical campus' status by all india council for technical education </triple> <triple> all india council for technical education location mumbai </triple> <triple> karnataka has to its northeast telangana </triple> <triple> visvesvaraya technological university city belgaum </triple> <triple> karnataka has to its west arabian sea </triple> <triple> acharya institute of technology state karnataka </triple> <end>
<start> the acharya institute of technology in karnataka state is affiliated with visvesvaraya technological university in belgium . it was given its technical campus' status by all india council for technical education in mumbai . karnataka has the arabian sea to its west and telagana to its northeast . <end>
[1]


In [38]:
probe(251)

<start> <triple> aip advances editor vincent h . crespi , bernard s . gerstman , a . t . charlie johnson , masaaki tanaka , enge g . wang </triple> <end>
<start> aleksandre guruli played for fc dinamo batumi and the olympique lyonnais club whose homeground is the parc olympique lyonnais stadium . <end>
[0]


In [51]:
from transformer_discriminator import *


In [52]:

def TransformerDiscriminator2(vocab_size, maxlen = 500,
                             embed_dim = 32,  # Embedding size for each token
                             num_heads = 2 ,  # Number of attention heads
                             ff_dim = 32):    # # Hidden layer size in feed forward network inside transformer

    #vocab_size = tokenizer_txt.vocab_size+2  # Only consider the top 20k words
    #maxlen = 500  # Only consider the first 200 words of each movie review


    inputs_rdf = layers.Input(shape=(maxlen,), name='rdf')
    inputs_txt = layers.Input(shape=(maxlen,), name='txt')
    
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    
    x_rdf = embedding_layer(inputs_rdf)
    x_txt = embedding_layer(inputs_txt)

    
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    
    x_rdf = transformer_block(x_rdf)
    x_rdf = layers.GlobalAveragePooling1D()(x_rdf)
    x_rdf = layers.Dropout(0.1)(x_rdf)

    x_txt = transformer_block(x_txt)
    x_txt = layers.GlobalAveragePooling1D()(x_txt)
    x_txt = layers.Dropout(0.1)(x_txt)

    x = layers.concatenate([x_rdf, x_txt])

    x = layers.Dense(20, activation="relu")(x)
    x = layers.Dropout(0.1)(x)
    outputs = layers.Dense(1, activation="sigmoid", name='real_prob')(x)

    model = keras.Model(inputs=[inputs_rdf, inputs_txt], outputs=outputs)

    model.summary()


    return model


discriminator = TransformerDiscriminator2(tokenizer_txt.vocab_size+2, maxlen=135)

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
rdf (InputLayer)                [(None, 135)]        0                                            
__________________________________________________________________________________________________
txt (InputLayer)                [(None, 135)]        0                                            
__________________________________________________________________________________________________
token_and_position_embedding_1  (None, 135, 32)      344320      rdf[0][0]                        
                                                                 txt[0][0]                        
__________________________________________________________________________________________________
transformer_block_1 (Transforme (None, 135, 32)      6464        token_and_position_emb

## Train and Evaluate


In [70]:
discriminator.compile("adam", "binary_crossentropy", metrics=["accuracy"])



history = discriminator.fit(
    {"rdf": Xr_train,
     "txt": Xt_train},
    {"real_prob": y_train},
    epochs=2,
    batch_size=32,
    shuffle=True,
    validation_data=(
                      {"rdf": Xr_test,
                      "txt": Xt_test},
                     {"real_prob": y_test}
                    )
                )

Epoch 1/2
1933/1933 [==============================] - 163s 85ms/step - loss: 0.1888 - accuracy: 0.9345 - val_loss: 0.1814 - val_accuracy: 0.9412
Epoch 2/2
1933/1933 [==============================] - 178s 92ms/step - loss: 0.1474 - accuracy: 0.9516 - val_loss: 0.1696 - val_accuracy: 0.9460


In [ ]:
discriminator.save_weights('./discriminator_weights.h5')